In [2]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import pickle
from utils import *
import time

from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode
import zlib

import warnings
warnings.filterwarnings('ignore')

img = cv2.imread('beer.png')

In [3]:
root = '/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/cortices/'

R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

C = Cortex(gpu=False)
C.loadLocs(root+'50k_Lloc_tight.pkl', root+'50k_Rloc_tight.pkl')
C.loadCoeffs(root+'50k_Lcoeff_tight.pkl', root+'50k_Rcoeff_tight.pkl')

In [11]:
def decode_data(data):
    encoded_V = data[0]
    code_dict = data[1]
    bits = data[2]
    
    V = huffman_decode(encoded_V, symbol_code_dict=code_dict, stream_length=bits)
    return V.astype('uint8')

In [14]:
import zmq
context = zmq.Context()

print("Connecting to Client…")
client_socket = context.socket(zmq.PULL)
client_socket.bind("tcp://*:5555")

while True:
    data_type = client_socket.recv()
    
    ret, cort = False, False
    if data_type == b'Ret':
        ret = True
    elif data_type == b'Cort':
        cort = True
    
    decoding_times = []
    backprojection_times = []
   
    start = time.time()
    while True:
        data = client_socket.recv()
        if data == b'Done':
            break
        
        t1 = time.time()
        
        decompressed_data = zlib.decompress(data)
        encoded_data = pickle.loads(decompressed_data) 
        V = decode_data(encoded_data)
        
        t2 = time.time()
        
        if ret:
            V = V.reshape((50000, 3))
            image = R.backproject_tight(V, (1080,1920,3), fixation)
        elif cort:
            V = V.reshape((50000, 3))
            image = C.cort_img(V)
        else:
            image = V.reshape((1080, 1920, 3))
        
        t3 = time.time()
        
        decoding_times.append(t2-t1)
        backprojection_times.append(t3-t2)
        
        cv2.imshow('Image', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
    end = time.time()
    client_time = float(client_socket.recv())
    server_time = end-start
    
    cv2.destroyWindow('Image')
    cv2.waitKey(1)
    
    print("----------Report----------")
    print("Total time: %f"%(client_time+server_time))
    print("Client side total time: %f"%(client_time))
    print("Server side total time: %f"%(server_time))
    
    print("Server side decoding times: %f"%sum(decoding_times))
    print("Server side vector to image times: %f"%sum(backprojection_times))
    print("-"*30)
    print()

    

Connecting to Client…
----------Report----------
Total time: 107.960884
Client side total time: 25.995316
Server side total time: 81.965568
Server side decoding times: 74.591167
Server side vector to image times: 0.002037
------------------------------

----------Report----------
Total time: 67.671174
Client side total time: 44.274118
Server side total time: 23.397056
Server side decoding times: 1.403816
Server side vector to image times: 21.914256
------------------------------



KeyboardInterrupt: 

In [ ]:
video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()

count = 1
while success:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if count ==1:
        print(img.shape)
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success, img = vidcap.read()
    count += 1

cv2.destroyWindow('Image')
cv2.waitKey(1)
vidcap.release()